#                          Web Scraping Assignment - 3

# Importing necessary libraries

In [1]:
import pandas as pd
import selenium
from bs4 import BeautifulSoup
import time
from selenium import webdriver
import requests
import re
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException, StaleElementReferenceException
from selenium.webdriver.support.ui import WebDriverWait

import warnings
warnings.filterwarnings('ignore')

# Q1:  Write a python program which searches all the product under a particular product vertical from www.amazon.in. The product verticals to be searched will be taken as input from user. For e.g. If user input is ‘guitar’. Then search for guitars.

In [12]:
# connecting to the webdriver
driver=webdriver.Chrome(r"chromedriver.exe")

In [13]:
# getting the webpage of mentioned url
url = "https://www.amazon.in/"
driver.get(url)

In [14]:
# entering the product that we want to search
user_input = input('Enter the product that we want to search : ')

Enter the product that we want to search : guitar


In [15]:
# searching the web element for user input
search = driver.find_element(By.ID,"twotabsearchtextbox")
search

# sending the user input to search bar
search.send_keys(user_input)

# locating the search button using xpath
search_btn = driver.find_element(By.XPATH,"//div[@class='nav-search-submit nav-sprite']/span/input")

# clicking on search button
search_btn.click()

# Q2 : In the above question, now scrape the following details of each product listed in first 3 pages of your search results and save it in a data frame and csv. In case if any product vertical has less than 3 pages in search results then scrape all the products available under that product vertical. Details to be scraped are: "Brand Name", "Name of the Product", "Rating", "No. of Ratings", "Price", "Return/Exchange", "Expected Delivery", "Availability", "Other Details" and “Product URL”. In case, if any of the details are missing for any of the product then replace it by “-“.

In [18]:
# fetching URLs to open the pages
urls = []          # empty list
for i in range(0,3):      # for loop to scrape 3 pages
    page_url = driver.find_elements(By.XPATH,"//a[@class='a-link-normal s-no-outline']")
    for i in page_url:
        urls.append(i.get_attribute("href"))
    next_btn = driver.find_element(By.XPATH,"/html/body/div[1]/div[2]/div[1]/div[1]/div/span[1]/div[1]/div[68]/div/div/span/a[3]")
    time.sleep(3)

In [19]:
len(urls)

216

In [22]:
# making empty list and fetching required data
brand_name = []
product_name = []
ratings = []
num_ratings = []
prices = []
exchange = []
exp_delivery = []
availability = []
other_details = []

for i in urls:
    driver.get(i)
    time.sleep(3)
    
    
    #fetching brand name 
    try:
        brand = driver.find_element(By.XPATH,"//a[@id='bylineInfo']")
        brand_name.append(brand.text)
    except NoSuchElementException:
        brand_name.append('-')
    
    
    # fetching Name of the Product
    try:
        product = driver.find_element(By.XPATH,"//span[@id='productTitle']")
        product_name.append(product.text)
    except NoSuchElementException:
        product_name.append('-')
        
        

     #fetching ratings
    try:
        rating = driver.find_element(By.XPATH,"//span[@class='a-size-base a-nowrap']/span")
        ratings.append(rating.text)
    except NoSuchElementException:
        ratings.append('-')
        
 
    #fetching  no of ratings
    try:
        num_rating = driver.find_element(By.XPATH,"//span[@id='acrCustomerReviewText']")
        num_ratings.append(num_rating.text)
    except NoSuchElementException:
        num_ratings.append('-')
        

    #fetching price of the product
    try:
        price = driver.find_element(By.XPATH,"//td[@class='a-span12']")
        prices.append(price.text)
    except NoSuchElementException:
        prices.append('-')
        
        
    #fetching return/exchange
    try:
        exch = driver.find_element(By.XPATH,"//span[@class='a-declarative']/div/a")
        exchange.append(exch.text)
    except NoSuchElementException:
        exchange.append('-')
        

    #fetching expected delivery
    try:
        delivery = driver.find_element(By.XPATH,"//div[@class='a-section a-spacing-mini']/b")
        exp_delivery.append(delivery.text)
    except NoSuchElementException:
        exp_delivery.append('-')
        

    #fetching availability information
    try:
        avail = driver.find_element(By.XPATH,"//span[@class='a-size-medium a-color-success']")
        availability.append(avail.text)
    except NoSuchElementException:
        availability.append('-')
        
    #other details
    try:
        oth_det = driver.find_element(By.XPATH,"//ul[@class='a-unordered-list a-vertical a-spacing-mini']")
        other_details.append(oth_det.text)
    except NoSuchElementException:
        other_details.append('-')
        

In [23]:
print(len(brand_name),
len(product_name),
len(ratings),
len(num_ratings),
len(prices),
len(exchange),
len(exp_delivery),
len(availability),
len(other_details))

216 216 216 216 216 216 216 216 216


In [24]:
# Creating the DataFrame for the scraped data

guitar = pd.DataFrame({})
guitar['Brand Name'] = brand_name
guitar['Name of the Product'] = product_name
guitar['Rating'] = ratings
guitar['No. of Ratings'] = num_ratings
guitar['Price'] = prices
guitar['Return/Exchange'] = exchange
guitar['Expected Delivery'] = exp_delivery
guitar['Availability'] = availability
guitar['Other Details'] = other_details
guitar['Product URL'] = urls
guitar

,Brand Name,Name of the Product,Rating,No. of Ratings,Price,Return/Exchange,Expected Delivery,Availability,Other Details,Product URL
0,Visit the Kadence Store,Kadence Frontier guitar with Online Guitar lea...,4 out of 5,"1,811 ratings","₹4,999.00",7 days Replacement,-,,"★ Design - The High Gloss finish, Simple head ...",https://www.amazon.in/sspa/click?ie=UTF8&spc=M...
1,Visit the Kadence Store,Kadence Slowhand Premium Jumbo Semi Acoustic G...,4 out of 5,538 ratings,"₹9,999.00",7 days Replacement,-,,ACOUSTIC-ELECTRIC GUITAR: Kadence Slowhand sem...,https://www.amazon.in/sspa/click?ie=UTF8&spc=M...
2,Visit the VAULT Store,Vault DA20 Dreadnought Acoustic Guitar With Gi...,4 out of 5,1 rating,-,7 days Returnable,-,In stock.,The Vault DA20 Dreadnought Acoustic Guitar is ...,https://www.amazon.in/sspa/click?ie=UTF8&spc=M...
3,Visit the blueberry Store,Blueberry B-38C-Brown 38 Inch Acoustic Guitar ...,-,-,-,7 days Returnable,-,In stock.,"Material: Top: linden wood, Side & Back: - lin...",https://www.amazon.in/sspa/click?ie=UTF8&spc=M...
4,Visit the JUAREZ Store,"Juârez Acoustic Guitar, 38 Inch Cutaway, 038C ...",4 out of 5,"18,158 ratings","₹2,399.00",7 days Replacement,-,In stock.,"Black Glossy Finish, Number of Frets: 18, Acou...",https://www.amazon.in/Juarez-Acoustic-Cutaway-...
...,...,...,...,...,...,...,...,...,...,...
211,Brand: Valencia,"Valencia Classical Guitar, Natural, 4/4 - VC404NA",4.8 out of 5,5 ratings,-,7 days Replacement,-,Only 1 left in stock.,Make sure this fits by entering your model num...,https://www.amazon.in/Valencia-Classical-Guita...
212,Visit the JUAREZ Store,"Juârez Acoustic Guitar, 38 Inch Cutaway, 038C ...",4 out of 5,"18,158 ratings","₹2,399.00",7 days Replacement,-,In stock.,"Black Glossy Finish, Number of Frets: 18, Acou...",https://www.amazon.in/Juarez-Acoustic-Cutaway-...
213,Visit the Intern Store,"Intern INT-38C Right hand Acoustic Guitar Kit,...",4 out of 5,"12,016 ratings",-,7 days Replacement,-,In stock.,Great looks with an innovative design to produ...,https://www.amazon.in/Intern-INT-38C-Acoustic-...
214,Visit the Intern Store,Intern INT-38C Acoustic Guitar Kit (Blue),4 out of 5,"12,016 ratings","₹2,399.00",7 days Replacement,-,In stock.,Great looks with an innovative design to produ...,https://www.amazon.in/Intern-INT-38C-Acoustic-...


In [25]:
#saving the data in csv
guitar.to_csv("Guitar.csv")

In [26]:
driver.close()

# Q4 :  Write a python program to search for a smartphone(e.g.: Oneplus Nord, pixel 4A, etc.) on www.flipkart.com and scrape following details for all the search results displayed on 1st page. Details to be scraped: “Brand Name”, “Smartphone name”, “Colour”, “RAM”, “Storage(ROM)”, “Primary Camera”, “Secondary Camera”, “Display Size”, “Display Resolution”, “Processor”, “Processor Cores”, “Battery Capacity”, “Price”, “Product URL”. Incase if any of the details is missing then replace it by “- “. Save your results in a dataframe and CSV.

In [2]:
# connecting to the webdriver
driver=webdriver.Chrome(r"chromedriver.exe")

In [5]:
# getting the webpage of mentioned url
url = "https://www.flipkart.com/"
driver.get(url)

In [7]:
# # closing login popup button
# lonin_x_btn = driver.find_element(By.XPATH,"//div[@class='_2QfC02']//button").click()

In [8]:
# search for web element
search_bar = driver.find_element(By.XPATH,"//input[@class='_3704LK']")

# sending keys to search product
search_bar.send_keys("pixel 4A")

In [9]:
# location the search button using xpath
search_btn = driver.find_element(By.XPATH,"//button[@class='L0Z3Pu']")

# clicking on search button
search_btn.click()

In [10]:
# fetching 1st page of URLs of smartphone
page1_url = []
urls = driver.find_elements(By.XPATH,"//a[@class='_1fQZEK']")
for url in urls:
    page1_url.append(url.get_attribute('href'))

In [11]:
len(page1_url)

24

In [12]:
# creating empty list
Smartphones = {}
Smartphones['Brand'] = []
Smartphones['Phone name'] = []
Smartphones['Colour'] = []
Smartphones['RAM'] = []
Smartphones['Storage(ROM)'] = []
Smartphones['Primary Camera'] = []
Smartphones['Secondary Camera'] = []
Smartphones['Display Size'] = []
Smartphones['Display Resolution'] = []
Smartphones['Processor'] = []
Smartphones['Processor Cores'] = []
Smartphones['Battery Capacity'] = []
Smartphones['Price'] = []
Smartphones['URL'] = []

In [17]:
# scraping data from each url of page 1
for url in page1_url:
    driver.get(url)
    print("Scraping URL = ",url)
    Smartphones['URL'].append(url)
    time.sleep(2)
    
    
    #clicking on read more button to get more information
    try:
        read_more = driver.find_element(By.XPATH,"//button[@class='_2KpZ6l _1FH0tX']")
        read_more.click()
    except NoSuchElementException:
        print("Exception occured while moving to next page")
    
    #scraping brand name of smartphone
    try:
        brand_tags = driver.find_element(By.XPATH,"//span[@class='B_NuCI']")
        Smartphones['Brand'].append(brand_tags.text.split()[0])
    except NoSuchElementException:
        Smartphones['Brand'].append('-')
    
    
    # scraping name of smartphones
    try:
        name_tags = driver.find_element(By.XPATH,"//div[@class='_3k-BhJ'][1]/table/tbody/tr[3]/td[2]/ul/li")
        Smartphones['Phone name'].append(name_tags.text)
    except NoSuchElementException:
        Smartphones['Phone name'].append('-')
        
    #scraping colour of smartphone
    try:
        color_tags = driver.find_element(By.XPATH,"//div[@class='_3k-BhJ'][1]/table/tbody/tr[4]/td[2]/ul/li")
        Smartphones['Colour'].append(color_tags.text)
    except NoSuchElementException:
        Smartphones['Colour'].append('-')
        
    # scraping RAM data of smartphone
    try:
        ram_tags = driver.find_element(By.XPATH,"//div[@class='_3k-BhJ'][4]/table[1]/tbody/tr[2]/td[2]/ul/li")
        Smartphones['RAM'].append(ram_tags.text)
    except NoSuchElementException:
        Smartphones['RAM'].append('-')
        
    #scraping ROM data of smartphones
    try:
        rom = driver.find_element(By.XPATH,"//div[@class='_3k-BhJ'][4]/table[1]/tbody/tr[1]/td[2]/ul/li")
        Smartphones['Storage(ROM)'].append(rom.text)
    except NoSuchElementException:
        Smartphones['Storage(ROM)'].append('-')
        
    # scraping  Primary camera data of smartphone
    try:
        pri =driver.find_element(By.XPATH,"//div[@class='_3k-BhJ'][5]/table[1]/tbody/tr[2]/td[2]/ul/li")
        Smartphones['Primary Camera'].append(pri.text)
    except NoSuchElementException:
        Smartphones['Primary Camera'].append('-')
        
    # scraping secondary camera data of smartphone
    try:
        sec = driver.find_element(By.XPATH,"//div[@class='_3k-BhJ'][5]/table[1]/tbody/tr[6]/td[1]")
        if sec != 'Secondary Camera' :
            if driver.find_element(By.XPATH,"//div[@class='_3k-BhJ'][5]/table[1]/tbody/tr[5]/td[1]").text == "Secondary Camera":
                sec_cam =driver.find_element(By.XPATH,"//div[@class='_3k-BhJ'][5]/table[1]/tbody/tr[5]/td[2]/ul/li")
            else :
                raise NoSuchElementException
        else :
            sec_cam = driver.find_element(By.XPATH,"//div[@class='_3k-BhJ'][5]/table[1]/tbody/tr[6]/td[2]/ul/li")
        Smartphones['Secondary Camera'].append(sec_cam.text)
    except NoSuchElementException:
        Smartphones['Secondary Camera'].append('-')
        
    
    #scraping display size data of smartphone
    try:
        disp = driver.find_element(By.XPATH,"//div[@class='_3k-BhJ'][2]/div")
        if disp.text != 'Display Features' : raise NoSuchElementException
        disp_size = driver.find_element(By.XPATH,"//div[@class='_3k-BhJ'][2]/table[1]/tbody/tr[1]/td[2]/ul/li")
        Smartphones['Display Size'].append(disp_size.text)
    except NoSuchElementException:
        Smartphones['Display Size'].append('-')
        
    
    #scraping display resolution of smartphone
    try:
        disp = driver.find_element(By.XPATH,"//div[@class='_3k-BhJ'][2]/div")
        if disp.text != 'Display Features' : raise NoSuchElementException
        disp_reso = driver.find_element(By.XPATH,"//div[@class='_3k-BhJ'][2]/table[1]/tbody/tr[2]/td[2]/ul/li")
        Smartphones['Display Resolution'].append(disp_reso.text)
    except NoSuchElementException:
        Smartphones['Display Resolution'].append('-')
        
        
    #scraping processor of smartphone
    try:
        pro = driver.find_element(By.XPATH,"//div[@class='_3k-BhJ'][3]/table[1]/tbody/tr[2]/td[1]]")
        if pro.text != 'Processor Type' : raise NoSuchElementException
        processor = driver.find_element(By.XPATH,"//div[@class='_3k-BhJ'][3]/table[1]/tbody/tr[2]/td[2]/ul/li")
        Smartphones['Processor'].append(processor.text)
    except NoSuchElementException:
        Smartphones['Processor'].append('-')
    
        
       
    # scraping processor core of smartphone
    try:
        core = driver.find_element(By.XPATH,'//*[@id="container"]/div/div[3]/div[1]/div[2]/div[9]/div[5]/div/div[2]/div[1]/div[3]/table/tbody/tr[2]/td[2]/ul/li')
        if core.text != 'Processor Core' :
            core = driver.find_element(By.XPATH,"//div[@class='_3k-BhJ'][3]/table[1]/tbody/tr[2]/td[1]")
            if core.text != 'Processor Core' :
                raise NoSuchElementException
            else :
                cores = driver.find_element(By.XPATH,"//div[@class='_3k-BhJ'][3]/table[1]/tbody/tr[2]/td[2]/ul/li")
        else :
            cores = driver.find_element(By.XPATH,"//div[@class='_3k-BhJ'][3]/table[1]/tbody/tr[3]/td[2]/ul/li")
        Smartphones['Processor Cores'].append(disp_reso.text)
    except NoSuchElementException:
        Smartphones['Processor Cores'].append('-')
        
        
        
    # scraping the battery capacity of smartphone
    try:
        if driver.find_element(By.XPATH,"//div[@class='_3k-BhJ'][10]/div").text != "Battery & Power Features" :
            if driver.find_element(By.XPATH,"//div[@class='_3k-BhJ'][9]/div").text == "Battery & Power Features" :
                bat_tags = driver.find_element(By.XPATH,"//div[@class='_3k-BhJ'][9]/table/tbody/tr/td[1]")
                if bat_tags.text != "Battery Capacity" : raise NoSuchElementException
                bat_capa = driver.find_element(By.XPATH,"//div[@class='_3k-BhJ'][9]/table/tbody/tr/td[2]/ul/li")
            elif driver.find_element(By.XPATH,"//div[@class='_3k-BhJ'][8]/div").text == "Battery & Power Features" :
                bat_tags = driver.find_element(By.XPATH,"//div[@class='_3k-BhJ'][8]/table/tbody/tr/td[1]")
                if bat_tags.text != "Battery Capacity" : raise NoSuchElementException
                bat_capa = driver.find_element(By.XPATH,"//div[@class='_3k-BhJ'][8]/table/tbody/tr/td[2]/ul/li")
            else:
                raise NoSuchElementException
        else :
            bat_tags = driver.find_element(By.XPATH,"//div[@class='_3k-BhJ'][10]/table/tbody/tr/td[1]")
            if bat_tags.text != "Battery Capacity" : raise NoSuchElementException
            bat_capa = driver.find_element(By.XPATH,"//div[@class='_3k-BhJ'][10]/table/tbody/tr/td[2]/ul/li")
        Smartphones['Battery Capacity'].append(bat_capa.text)
    except NoSuchElementException:
        Smartphones['Battery Capacity'].append('-')
    
    
    
    
    # scraping price of smartphone
    try:
        price_tags = driver.find_element(By.XPATH,"//div[@class='_30jeq3 _16Jk6d']")
        Smartphones['Price'].append(price_tags.text)
    except NoSuchElementException:
          Smartphones['Price'].append('-')         

TimeoutException: Message: timeout: Timed out receiving message from renderer: 600.000
  (Session info: chrome=108.0.5359.72)
Stacktrace:
Backtrace:
	(No symbol) [0x004EF243]
	(No symbol) [0x00477FD1]
	(No symbol) [0x0036D04D]
	(No symbol) [0x0035FC24]
	(No symbol) [0x0035FA5C]
	(No symbol) [0x0035E7E8]
	(No symbol) [0x0035EDC2]
	(No symbol) [0x0035DC55]
	(No symbol) [0x0036899C]
	(No symbol) [0x0035D735]
	(No symbol) [0x0035F703]
	(No symbol) [0x0035E7E8]
	(No symbol) [0x0035EDC2]
	(No symbol) [0x0035DC55]
	(No symbol) [0x00366493]
	(No symbol) [0x0035D735]
	(No symbol) [0x0035F703]
	(No symbol) [0x0035E7E8]
	(No symbol) [0x0035EDC2]
	(No symbol) [0x0035DC55]
	(No symbol) [0x00364072]
	(No symbol) [0x0035D735]
	(No symbol) [0x0035F703]
	(No symbol) [0x0035E7E8]
	(No symbol) [0x0035EDC2]
	(No symbol) [0x0035DC55]
	(No symbol) [0x00357E66]
	(No symbol) [0x0035D735]
	(No symbol) [0x0035CEE0]
	(No symbol) [0x0035CFD5]
	(No symbol) [0x0035D388]
	(No symbol) [0x0035D1F8]
	(No symbol) [0x003

In [13]:
# checking lengths of all scraped data

print(len(Smartphones['Brand']),len(Smartphones['Phone name']), len(Smartphones['Colour']),len(Smartphones['RAM']),len(Smartphones['Storage(ROM)']),len(Smartphones['Primary Camera']),len(Smartphones['Secondary Camera']), len(Smartphones['Display Size']), len(Smartphones['Display Resolution']), len(Smartphones['Processor']), len(Smartphones['Processor Cores']), len(Smartphones['Battery Capacity']), len(Smartphones['Price']), len(Smartphones['URL'])) 

0 0 0 0 0 0 0 0 0 0 0 0 0 0


In [ ]:
# framing the DataFrame

df = pd.DataFrame.from_dict(Smartphones)
df

In [ ]:
# saving the data in csv
df.to_csv("smartphones.csv")

In [ ]:
driver.close()

# Q5 : Write a program to scrap geospatial coordinates (latitude, longitude) of a city searched on google maps.

In [ ]:
# connecting to the webdriver
driver=webdriver.Chrome(r"chromedriver.exe")

In [ ]:
# getting mentioned url and opening google maps web page
url = 'https://www.google.co.in/maps'
driver.get(url)
time.sleep(2)

In [ ]:
# entering the city name in search bar
City = input('Enter City name that has to be searched : ')
search_bar = driver.find_element_by_id('searchboxinput')
search_bar.click()
time.sleep(2)

#sending keys to find cities
search_bar.send_keys(City)

#checking for webelement and clicking on search button
search_btn = driver.find_element_by_id("searchbox-searchbutton")
search_btn.click()
time.sleep(2)

try:
    url_str = driver.current_url
    print("URL Extracted: ", url_str)
    latitude_longitude = re.findall(r'@(.*)data',url_str)
    if len(latitude_longitude):
        lat_lng_list = latitude_longitude[0].split(",")
        if len(lat_lng_list)>=2:
            latitude = lat_lng_list[0]
            longitude = lat_lng_list[1]
        print("Latitude = {}, Longitude = {}".format(latitude, longitude))
except Exception as e:
        print("Error: ", str(e))

In [ ]:
driver.close()

# Q7 :  Write a program to scrap all the available details of best gaming laptops from digit.in.

In [44]:
# connecting to the webdriver
driver=webdriver.Chrome(r"chromedriver.exe")

In [45]:
# opening the url digit.in
url = "https://www.digit.in/"
driver.get(url)
time.sleep(2)

In [46]:
# searching for best Laptop
best_gam_laptops = driver.find_element(By.XPATH,"//div[@class='listing_container']//ul//li[9]").click()
time.sleep(3)

In [47]:
# creating empty list
Laptop_Name = []
Operating_sys = []
Display = []
Processor = []
Memory = []
Weight = []
Dimensions = []
Graph_proc = []
Price = []

In [48]:
#scraping the data of laptop names
laptop_name = driver.find_elements(By.XPATH,'//div[@class="left_side"]')
for name in laptop_name:
    Laptop_Name.append(name.get_attribute('a'))
    
#scraping the data of operating system
try:
    op_sys = driver.find_elements(By.XPATH,"//div[@class='product-detail']/div/ul/li[1]/div/div")
    for os in op_sys:
        Operating_sys.append(os.text)
except NoSuchElementException:
    pass


#scraping data of display of the Laptop
try:
    display = driver.find_elements(By.XPATH,"//div[@class='product-detail']/div/ul/li[2]/div/div")
    for disp in display:
        Display.append(disp.text)
except NoSuchElementException:
    pass


# scraping data of processor
try:
    processor = driver.find_elements(By.XPATH,"//div[@class='Spcs-details'][1]/table/tbody/tr[5]/td[3]")
    for pro in processor:
        Processor.append(pro.text)
except NoSuchElementException:
    pass


# scraping the data of memory
try:
    memory = driver.find_elements(By.XPATH,"//div[@class='Spcs-details'][1]/table/tbody/tr[6]/td[3]")
    for memo in memory:
        Memory.append(memo.text)
except NoSuchElementException:
    pass


# scraping data of weight
try:
    weight = driver.find_elements(By.XPATH,"//div[@class='Spcs-details'][1]/table/tbody/tr[7]/td[3]")
    for wgt in weight:
        Weight.append(wgt.text)
except NoSuchElementException:
    pass


# scraping data of dimensions
try:
    dimension = driver.find_elements(By.XPATH,"//div[@class='Spcs-details'][1]/table/tbody/tr[8]/td[3]")
    for dim in dimension:
        Dimensions.append(dim.text)
except NoSuchElementException:
    pass


# scraping data of graph processor
try:
    graph = driver.find_elements(By.XPATH,"//div[@class='Spcs-details'][1]/table/tbody/tr[9]/td[3]")
    for gra in graph:
        Graph_proc.append(gra.text)
except NoSuchElementException:
    pass


# scraping the data of price
try:
    price = driver.find_elements(By.XPATH,"//td[@class='smprice']")
    for pri in price:
        Price.append(pri.text.replace('₹ ','Rs'))
except NoSuchElementException:
    pass


In [49]:
print(len(Laptop_Name),
len(Operating_sys),
len(Display),
len(Processor),
len(Memory),
len(Weight),
len(Dimensions),
len(Graph_proc),
len(Price))

11 10 10 10 10 10 10 9 10


In [40]:
#creating DataFrame for scraped data
Gaming_Laptop=pd.DataFrame({})
Gaming_Laptop['Laptop Name'] = Laptop_Name
Gaming_Laptop['Operating System'] =Operating_sys
Gaming_Laptop['Display'] = Display
Gaming_Laptop['Processor'] = Processor
Gaming_Laptop['Memory'] = Memory
Gaming_Laptop['Weight'] = Weight
Gaming_Laptop['Dimensions'] = Dimensions
Gaming_Laptop['Graphical Processor'] = Graph_proc
Gaming_Laptop['Price'] = Price
Gaming_Laptop

ValueError: Length of values (10) does not match length of index (11)

In [ ]:
# saving the data to csv
Gaming_Laptop.to_csv("Gaming_Laptops.csv")

In [33]:
driver.close()